<h1><center>Laboratorio 5: La desperación de Mr. Cheems 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Lucas Orellana J.
- Nombre de alumno 2: Elizabeth Ramírez Z.


### **Link de repositorio de GitHub:** [Repositorio](https://github.com/ElLuquitas/MDS7202-labs.git)

## Temas a tratar
- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines y Column Transformers

## Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Jueves a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio
- Comprender cómo aplicar pipelines de Scikit-Learn para generar procesos más limpios en Feature Engineering.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
# !pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except:
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


# Feature engineering en datos de retail 🛍️

### 0. Cargar Dataset

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

Mr. Cheems, gerente de una cotizada tienda de retail en Europa, les solicita si pueden analizar los datos de algunas de sus tiendas. En una reunión, Mr Cheems le comenta que la calidad de sus datos no es muy buena, por lo que le solicita a usted que limpie su base de datos y cree nuevos atributos relevantes para el negocio.

Por ello, el área de ventas les entrega archivo llamado `online_retail_data.pickle` el cual usted decide cargar a continuación.

In [3]:
# Inserte su código aquí
df_retail = pd.read_pickle('online_retail_data.pickle')
df_retail.head()

Invoice StockCode                          Description  Quantity  \
0  ID489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS      12.0   
1  ID489434    79323P                   PINK CHERRY LIGHTS      12.0   
2  ID489434    79323W                  WHITE CHERRY LIGHTS      12.0   
3  ID489434     22041         RECORD FRAME 7" SINGLE SIZE       48.0   
4  ID489434     21232       STRAWBERRY CERAMIC TRINKET BOX      24.0   

          InvoiceDate  Price Customer ID         Country  
0 2009-12-01 07:45:00   6.95     13085.0  United Kingdom  
1                 NaT   6.75     13085.0  United Kingdom  
2 2009-12-01 07:45:00   6.75     13085.0  United Kingdom  
3 2009-12-01 07:45:00   2.10     13085.0  United Kingdom  
4 2009-12-01 07:45:00   1.25     13085.0  United Kingdom

### 1. Función para explorar características [5 puntos]

<p align="center">
  <img width=300 src="https://editor.analyticsvidhya.com/uploads/47389meme.png">
</p>




Tras inspeccionar brevemente los datos proporcionados, usted decide crear una función que realice lo siguiente:
- Plotee un histograma para las variables precios y cantidad. [3 puntos]
- Imprima un conteo de datos nulos por variable [2 puntos]

NOTA: Para generar los gráficos es **OBLIGATORIO** el uso de plotly.

**Respuesta:**

In [4]:
def explore_data(dataframe_in, column_name = ['Price', 'Quantity']):
    """
    Función que permite:
    - Hacer plot de histogramas para las variables `Price` y `Quantity`.
    - Mostrar la cantidad de nulos por cada variable del dataset.

    Args:
      dataframe_in: DataFrame de pandas.
      column_name: Nombre de la o las columnas a analizar.
    """

    # Si hay una sola columna, graficarla
    if isinstance(column_name, str):
        fig = px.histogram(dataframe_in, x=column_name, title=f'Conteo de {column_name}')
        fig.show()
        return
    
    for col in column_name:
        fig = px.histogram(dataframe_in, x=col, title=f'Conteo de {col}')
        fig.show()

    # Ahora mostramos los valores nulos
    print('\n Cantidad de nulos por variable:')
    print(dataframe_in.isnull().sum())

In [ ]:
#Implementar la función 
explore_data(df_retail)

### 2. Eliminando outliers [10 puntos]

<p align="center">
  <img width=300 src="https://media.licdn.com/dms/image/C5612AQGdXKCka7HumA/article-cover_image-shrink_600_2000/0/1520056407281?e=2147483647&v=beta&t=VZcfjjzjK4LxXdZkSu1KisWC0Ry8bk4tPCn3R8aYdNM">
</p>




#### 2.1 Creando la clase IQR [5 puntos]

Entre las falencias de los datos, Mr. Cheems le comenta que a veces los operadores no ingresan el precio correcto de los productos. Mr. Cheems le comenta que se dio cuenta de este fenómeno porque hay productos con precios exagerádamente altos o bajos. Por lo cual usted decide eliminar outliers del dataframe a traves del rango intercuartil el cual cuenta con los siguientes pasos:

1. Calcular el primer cuartil $Q1$ y el tercer cuartil $Q3$. Hint: utilice el método `quantile()`

2. Calcular el rango intercuartil (RIC): $RIC = Q3 - Q1$

3. Calcular los límites para identificar outliers:
 - Límite inferior: $~~Q1 - \lambda \cdot RIC$
 - Límite superior: $~~Q3 + \lambda \cdot RIC$

4. Eliminar outliers: Los outliers son los datos que están por debajo del límite inferior o por encima del límite superior.


Para realizar dicha tarea, usted decide crear una clase llamada `IQR()` utilizando `BaseEstimator` y `TransformerMixin` para realizar una transformación de cada una de las columnas numéricas del DataFrame utilizando `ColumnTransformer()` más tarde. Considere que lambda debe ser $\lambda$ un parámetro a definir por el usuario.

Hint: tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer).
>**Nota: No modificar el método set_output de la clase IQR**

**Respuesta:**

A continuación un esquema de cómo se deben realizar los pasos para calcular los *outliers*. Tomamos como ejemplo la columna `Price`, pero se hará aplicable para cualquier columna numérica:

```py
# Calcular primer cuartil Q_1 y tercer cuartil Q_3 usando `quantile`
Q_1 = df_retail['Price'].quantile(0.25)
Q_3 = df_retail['Price'].quantile(0.75)

# Calcular rango intercuartil (RIC)
RIC = Q_3 - Q_1

# Definir los límites inferior y superior
lambda_param = 1.5
lower_bound = Q_1 - lambda_param * RIC
upper_bound = Q_3 + lambda_param * RIC

# Filtrar los datos
df_retail_filtered = df_retail[(df_retail['Price'] > lower_bound) & (df_retail['Price'] < upper_bound)]
```

In [6]:
class IQR(BaseEstimator, TransformerMixin):

  def __init__(self, lambda_param):
    self.lambda_param = lambda_param

  # Para el método `fit` se espera que hayan dos argumentos según el link del hint,
  # pero en este caso `y` se igualará a `None` para evitar posibles errores.
  def fit(self, X, y = None):
    Q_1 = X.quantile(0.25)
    Q_3 = X.quantile(0.75)
    RIC = Q_3 - Q_1
    self.lower_bound = Q_1 - self.lambda_param * RIC
    self.upper_bound = Q_3 + self.lambda_param * RIC

    return self

  def transform(self, X):
    # Para las transformaciones, trabajaremos sobre una copia de la columna
    X_definitive = X.copy()

    # Filtrar los datos
    X_definitive = X_definitive[(X_definitive > self.lower_bound) & (X_definitive < self.upper_bound)]

    return X_definitive

  def set_output(self, transform = 'default'):
    #No modificar esta función
    return self

#### 2.2 Creación del Pipeline [5 puntos]

Para comenzar introduciéndose en el uso de pipeline, usted decide definir un pipeline con el Transformer previamente definido. Además, usted decide visualizar cómo cambia la distribución de las variables Precio y Cantidad antes y despues de aplicar IQR. Para ello, usted aplica los siguientes pasos:

- Definir un pipeline llamado `numeric_transformations` para las variables precio y cantidad con la transformación IQR. [1 punto]
- Defina un column transformer que aplique `numeric_transformations` para las variables numéricas y `passthrough` para las variables categóricas. Adicionalmente, fije el parámetro `verbose_feature_names_out` en `False`. Ver hint al final [1 puntos]
- Defina el dataframe `df_iqr` aplicado el column transformer a los datos proporcionados por Mr. Cheems considerando un valor de $\lambda$ que tenga un desempeño aceptable para ambas variables. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_iqr`.  [1 punto]
- Reportar los cambios observados en la distribución de las variables. ¿Qué sucede al aumentar el valor de lambda? [1 punto]


HINT: El transformador `passthrough` está predefinido y es una opción que puedes usar para las columnas que no deseas transformar. Al especificar 'passthrough' para una parte de tu ColumnTransformer, las columnas correspondientes pasarán a través del ColumnTransformer sin ninguna modificación. El siguiente [enlace](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) le puede ser útil.
>**Nota: Mantenga el método set_output del column transformer con la transformación `pandas` para obtener un dataframe una vez aplicado el column transformer.**

**Respuesta:**

Apóyese de la siguiente estructura para su respuesta:

In [ ]:
# Definicion las variables que pasarán por cada pipeline
numerical_columns = ('Price', 'Quantity')
categorical_columns = ('Invoice', 'StockCode', 'Description', 'InvoiceDate', 'Customer ID', 'Country')

# Definicion del pipeline
numeric_transformations = Pipeline([('iqr', IQR(lambda_param = 1.5))])

# ColumnTransformer
column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', 'passthrough', categorical_columns)
                                        ],
                                        verbose_feature_names_out = False)

# Mantener el formato de pandas
column_transformer.set_output(transform='pandas')

# Aplicamos ColumnTransformer a los datos
df_iqr = column_transformer.fit_transform(df_retail)

# Gráficos
print('Datos originales:')
explore_data(df_retail)

print('\nDatos después de aplicar IQR:')
explore_data(df_iqr)

**Reportar los cambios observados en la distribución de las variables. ¿Qué sucede al aumentar el valor de lambda?**

Al hacer los gráficos de precio y cantidad con los datos entregados, se ve que hay *outliers* que hacen que los histogramas se vean con un rango amplio de datos. Al aplicar el filtro, podemos ver en los gráficos que las observaciones ya no están concentradas en la zona izquierda de cada histograma, existiendo una distribución más "uniforme". Los datos, tanto en cantidades como en precios, se concentraron en valores más pequeños en comparación a los primeros datos, acotándose los rangos entre los que varían los valores de las dos variables analizadas, siendo los nuevos conjuntos (aprox):

* `Price`: de $[0,0; 10000,0]$ a $[0,0; 7,5]$
* `Quantity`: de $[0; 18000]$ a $[0;26]$

Otro punto importante a notar es que la cantidad de nulos en las categorías `Price` y `Quantity` aumentan. Esto dado que los métodos aplicados no eliminan las filas en donde existen los *outliers*, sino que reemplazan estos valores por valores nulos. Estos valores nulos serán más o menos dependiendo del parámetro `lambda_param` que el usuario indique y del resultado que espera.

El valor de lambda indica cuánta tolerancia hay para detectar *outliers*. Básicamente, controla qué tan lejos de los cuartiles $Q_1$ y $Q_3$ pueden estar los datos antes de ser considerados como valores *outliers*. En el ejercicio, decidimos usar un $\lambda = 1.5$. Si aplicamos un $\lambda = 2.0$, el rango de valores aumenta. 
* `Price` con $\lambda = 2.0$: de $[0,0; 8,7]$
* `Quantity` con $\lambda = 2.0$: de $[1; 31]$ 


### 3. Agregando un imputer al pipeline [10 puntos]



<p align="center">
  <img width=300 src="https://media.makeameme.org/created/hmm-there-is.jpg">
</p>

Para continuar con la limpieza del dataframe usted decide imputar los datos nulos de las variables numéricas, para lo cual decide realizar las siguientes tareas:

1. Crear un pipeline para variables categóricas llamado `categoric_transformations` con un paso llamado `mode_imputer`, en el cual se imputen los datos faltantes por la categoría más frecuente.
2. Agregar al pipeline `numeric_transformations` un paso llamado `mean_imputer`, en el cual se imputen los datos por la media usando [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) [1 punto]
3. Crear y aplicar un `ColumnTransformer` actualizado con los pipelines `categoric_transformations` y `numeric_transformations` a `df_retail`, creando un dataframe llamado `df_mean_imputer`. [1 punto]
4. Comparar los resultados de `explore_data` en `df_mean_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
5. Cambiar el imputer de `numeric_transformations` por [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) y definir un nuevo dataframe llamado `df_knn_imputer`, aplicando el nuevo ColumnTransformer a `df_retail`. En caso de los tiempos de ejecución sean altos puede probar a reducir el parámetro `n_neighbors`. [1 punto]
6. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
7. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_mean_imputer`. ¿Cuál método de imputación es mejor? Deje el método escogido en el ColumnTransformer. [2 puntos]

>**Nota: Fije el parámetro verbose_feature_names_out en `False` y utilice el método set_output con transformación `pandas` en cada ColumnTransformer para obtener como salida un dataframe.**

**Respuesta:**

In [8]:
import time
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

# Pipeline para variables categóricas (standard)
categoric_transformation = Pipeline([('mode_imputer', SimpleImputer(strategy = 'most_frequent'))])

# Pipeline para variables numéricas (mean)
numeric_transformations_mean = Pipeline([('iqr', IQR(lambda_param = 1.5)),
                                    ('mean_imputer', SimpleImputer(strategy = 'mean'))])

# Pipeline para variables numéricas (KNN)
numeric_transformations_knn = Pipeline([('iqr', IQR(lambda_param = 1.5)),
                                    ('knn_imputer', KNNImputer(n_neighbors = 3))])

# ColumnTransformer usando mean
column_transformer_1 = ColumnTransformer([('numerical', numeric_transformations_mean, numerical_columns),
                                        ('categorical', categoric_transformation, categorical_columns)
                                        ],
                                        verbose_feature_names_out = False)

# ColumnTransformer usando KNN
column_transformer_2 = ColumnTransformer([('numerical', numeric_transformations_knn, numerical_columns),
                                        ('categorical', categoric_transformation, categorical_columns)
                                        ],
                                        verbose_feature_names_out = False)

# Mantener el formato de pandas
column_transformer_1.set_output(transform = 'pandas')
column_transformer_2.set_output(transform = 'pandas')

# Aplicamos ColumnTransformer a los datos
init_mean = time.time()
df_mean_imputer = column_transformer_1.fit_transform(df_retail)
end_mean = time.time()
print('Tiempo de ejecución para SimpleImputer con estrategia mean (en segundos):', end_mean - init_mean)

init_knn = time.time()
df_knn_imputer = column_transformer_2.fit_transform(df_retail)
end_knn = time.time()
print('Tiempo de ejecución para KNNImputer (en segundos):', end_knn - init_knn)

Tiempo de ejecución para SimpleImputer con estrategia mean (en segundos): 2.9319241046905518
Tiempo de ejecución para KNNImputer (en segundos): 1986.6127965450287


In [ ]:
# 1. Gráficos df_iqr 
print('Datos aplicando IQR:')
explore_data(df_iqr)

In [ ]:
#2. Gráficos df_mean_imputer 
print('\nDatos aplicando IQR + imputación media:')
explore_data(df_mean_imputer)

In [ ]:
#3. Gráficos df_knn_imputer 
print('\nDatos aplicando IQR + imputación KNN:')
explore_data(df_knn_imputer)

**Comparar los resultados de explore_data en df_mean_imputer y df_iqr. ¿Qué diferencias observa en la distribución de los datos?**

* La mayor diferencia es que en el caso de `df_iqr` hay una gran cantidad de datos nulos, ya que se filtraron los *outliers* sin llegar a eliminar las observaciones. 
* En la distribución de `Price` no se ve mayor diferencia, siendo estas mínimas. 
* En la distribución de `Quantity` sí se ve una diferencia, ya que aparecen valores decimales, ya que se imputa por la media y esta puede tomar valores decimales, entonces aparecen más barras en la distribución. Esto no tiene tanto sentido en esta variable, ya que la cantidad en los datos de entrada tenía valores de enteros. 

**Comparar los resultados de explore_data en df_knn_imputer y df_iqr. ¿Qué diferencias observa en la distribución de los datos?**

* Se observan diferencias. La primera es que `df_iqr` tiene valores nulos en el reporte que entrega la función `explore_data`, mientras que `df_knn_imputer` no tiene valores nulos. 
* Con respecto a la columna `Price`, se ve que los rangos de las barras del histograma son más acotados, mientras que en el KNN los rangos son mayores. La distribución se parece, solo que hay diferencias en los valores que fueron imputados con el método KNN. 
* Con respecto a los valores de `Quantity`, se ve que la distribución presenta mayores diferencias. Si bien la forma es similar, los valores imputados con KNN hacen que se formen  barras en el histograma más angostas, porque los valores imputados hacen que cambie un poco la distribución. 

**Comparar los resultados de explore_data en df_knn_imputer y df_mean_imputer. ¿Cuál método de imputación es mejor? Deje el método escogido en el ColumnTransforme.**

Con respecto a estos dos métodos, el método en que se obtiene el `df_mean_imputer` se demora bastante menos en ejecutarse que el método KNN en donde se obtiene el `df_knn_imputer`, como se pudo ver en el print del tiempo en segundos. 

Ambos métodos muestran que no hay valores nulos, ya que se realizó una imputación para su rellenado. 

En la variable de `Quantity`, no se nota grandemente la diferencia en la distribución de los datos, ya que ambos histogramas son bastante similares. 
En la variable `Price` se nota una mayor diferencia, ya que los rangos de precio en el histograma del KNN son más amplios, mientras que en el histograma de la imputación por la media los rangos con más acotados y hay más barras. La diferencia es que con la imputación con la media queda más variabilidad de valores, por esos se ve en el histograma barras más acotadas, pero más cantidad de barras en el rango entre $0$ a $7.5$, mientras que la imputación con KNN estima los valores faltantes basándose en los valores de las observaciones más cercanas y por eso se ven menos barras en total, pero con un ancho mayor, porque los datos faltantes los llena en base a los vecinos.
En resumen, la imputación con la media es más simple y rápido que el método KNN, pero no considera los valores de alrededor del dato faltante. Por su parte, la imputación con KNN, usa la similitud entre observaciones para estimar los valores faltantes, lo que lo hace más preciso, pero más costoso computacionalmente. 

Para terminar, no hay un método mejor, sino que depende de cada dataset. Si hay pocos datos faltantes, quizás usar la imputación por la media es recomendado, pero cuando faltan alta cantidad de datos, es mejor usar un método más sofisticado. Dado que en este caso, los datos faltantes son $40382$ para `Price` y $33779$ en `Quantity`, creemos que en mejor usar KNN aunque sea más costoso computacionalmente.

### 4. Creación de nuevas features [20 puntos]

<p align="center">
  <img width=250 src="https://miro.medium.com/max/1000/1*JtTWgAcfVTWV8OTjT47Atg.jpeg">
</p>


#### 4.1 Definicion de LRMFP [10 puntos]
(2 puntos por cada custom feature)

Dado que Mr. Lepin está interesado en obtener nuevos atributos relevantes para su negocio, su equipo de expertos sugiere la construcción de variables **LRMFP**, las que se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales. **Considere "hoy" como la fecha mas reciente del dataset**.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [12]:
def custom_features(dataframe_in):
    '''
    Función que permite crear un nuevo dataframe con las siguientes columnas:
    - Customer ID: Identificación del cliente.
    - Lenght: Intervalo de tiempo (días) entre la primera y última visita.
    - Recency: Días desde la última compra.
    - Frequency: Total de visitas del cliente durante un periodo de observación.
    - Monetary: Gasto medio del cliente por cada visita durante un periodo de observación.
    - Periodicity: Primero calcula la diferencia de días entre una visita y otra, luego se calcula la desviación estándar.

    Args:
        dataframe_in: DataFrame de pandas.

    Returns:
        DataFrame con las columnas descritas.
    '''

    # Creación de columna Lenght
    length_df = dataframe_in.groupby('Customer ID', observed = False)['InvoiceDate'].agg(Lenght = lambda x: (x.max() - x.min()).days)

    # Creación de columna Recency
    today = dataframe_in['InvoiceDate'].max()
    recency_df = dataframe_in.groupby('Customer ID', observed = False)['InvoiceDate'].agg(Recency = lambda x: (today - x.max()).days)

    # Creación de columna Frequency
    frequency_df = dataframe_in.groupby('Customer ID', observed = False)['Invoice'].count().rename('Frequency')

    # Creación de columna Monetary
    monetary_df = dataframe_in.groupby('Customer ID', observed = False)['Price'].mean().rename('Monetary')

    # Creación de columna Periodicity
    # Primero ordeno las fechas y luego calculo la diferencia de días entre una visita y otra
    periodicity_diff_df = dataframe_in.sort_values(by='InvoiceDate').groupby('Customer ID', observed = False)['InvoiceDate'].diff().dt.days
    periodicity_df = periodicity_diff_df.groupby(dataframe_in['Customer ID']).std().rename('Periodicity')

    # Concatenación de las columnas, haciendo merge con la columna 'Customer ID'
    df_custom = pd.concat([length_df, recency_df, frequency_df, monetary_df, periodicity_df], axis = 1)

    ### Algunos arreglos a las columnas ###
    # Redondeo de las columnas 'Monetary' y 'Periodicity' a 1 decimal
    df_custom['Monetary'] = df_custom['Monetary'].round(1)
    df_custom['Periodicity'] = df_custom['Periodicity'].round(1)

    # Rellenar los valores nulos en 'Periodicity'con 0 (si no hay visitas, la periodicidad es indefinida)
    df_custom['Periodicity'] = df_custom['Periodicity'].fillna(0)

    return df_custom

In [13]:
df_custom_features = custom_features(df_retail)
df_custom_features.head()

C:\Users\lucas\AppData\Local\Temp\ipykernel_14884\2204213945.py:34: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Lenght  Recency  Frequency  Monetary  Periodicity
Customer ID                                                   
12346.0       196.0    164.0         33       6.3         22.1
12347.0        37.0      2.0         71       2.2          4.5
12348.0         0.0     73.0         20       0.7          0.0
12349.0       181.0     42.0        102       8.6         16.3
12351.0         0.0     10.0         21       2.4          0.0

In [14]:
# Cantidad de nulos en df_custom
print('Cantidad de nulos en df_custom_features:')
print(df_custom_features.isnull().sum())

Cantidad de nulos en df_custom_features:
Lenght         3
Recency        3
Frequency      0
Monetary       2
Periodicity    0
dtype: int64


#### 4.2 Agregando las custom features [10 puntos]

Ahora, usted decide agregar al pipeline las nuevas variables creadas, para lo cual realiza las siguientes tareas:

1. Cree un nuevo pipeline llamado `retail_pipeline` que encapsule el ColumnTransformer y calcule las LRMFP. El primer paso del pipeline llámelo  `col_tranformer` y el segundo paso llámelo `custom_features`, incorpora las nuevas variables al dataframe. Hint: les puede ser útil investigar [este](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) método. [1 punto]
2. Aplicar el pipeline actualizado a los datos proporcionados por Mr. Cheems, creando un nuevo dataframe llamado `df_custom`. [1 punto]
3. Explorar la distribución de las nuevas variables con `explore_data` y comentar brevemente (2-3 líneas) características de cada custom feature. [5 puntos]
5. Entregar un insight para el negocio en base a las nuevas variables. [3 puntos]

>Nota: Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta**

In [15]:
# ColumnTransformer usando KNN
retail_pipeline = Pipeline([('col_transformer', column_transformer_2),
                            ('custom_features', FunctionTransformer(custom_features, validate = False))])

# Mantener el formato de pandas (viene de arriba, así que no hay problema)
init = time.time()
df_custom = retail_pipeline.fit_transform(df_retail)
end = time.time()
print('Tiempo de ejecución (en segundos):', end - init)

Tiempo de ejecución (en segundos): 1955.9095599651337


In [ ]:
# Exploración de los nuevos datos
explore_data(df_custom, df_custom.columns.to_list())

**Explorar la distribución de las nuevas variables con explore_data y comentar brevemente (2-3 líneas) características de cada custom feature.**

* La variable `Lenght` tiene una distribución con alto valores en el rango entre 0 a 9 (la mayor concentración de datos). Luego tiene dos concentraciones de datos hacia la derecha, una entre 200 a 290 y otra entre 320 a 380. 
* La variable `Recency` muestra una distribución como una logarítmica o gamma, con concentración de datos hacia el cero y una disminución alta de frecuencia hacia valores mayores. 
* La variable `Frecuency` también muestra una distribución como una logarítmica, con un peak en torno al 0 y una caída rápida hacia valores mayores. 
* La variable `Monetary` tiene una distribución similar a la normal, solo que tiene una pequeña asimetría, ya que hay más valores hacia la cola derecha que la izquierda. 
* La variable `Periodicity` tiene una alta concentración de datos entorno al cero y luego tiene una distribución que se pareciera a la normal pero con asimetría, ya que la cola derecha se prolonga mucho más que la cola izquierda. 

**Entregar un insight para el negocio en base a las nuevas variables.**

* Hay una gran cantidad de clientes que no son fieles clientes (sobre los 1000 clientes) que solo compraron 1 vez. Sin embargo, hay dos grupos de clientes que compran con más frecuencia: un grupo en torno a 250 días y otro entorno a 350 dias de fidelidad como clientes. 
* La mayor cantidad de clientes realizó compras recientemente, entre 0 a 30 días. La cantidad de clientes que no ha vuelto a comprar es menor en comparación a los clientes que compraron recientemente. 
* La frecuencia de los clientes tiende a estar concentrada en valores menores a 60 veces en el período analizado, por lo que los clientes no tienden a hacer compras tan seguido. 
* Con respecto al término monetario, la mayor parte de los clientes están en torno a 2.2 de contribución a la empresa con sus compras. Solo una menor cantidad sobrepasa el valor de 4. 
* Finalmente, la periodicidad de los clientes se concentra en torno a 0, es decir, no tienen periodicidad. Sin embargo hay un grupo de 250 clientes que se concentra en torno a 10 de periodicidad. 

### 5. MinMax Scaler [10 puntos]

<p align="center">
  <img width=300 src="https://i.imgflip.com/1fsprn.jpg">
</p>


#### 5.1 Definición del Column Transformer [5 puntos]

Construya una clase llamada `MinMax()` para realizar una transformación de cada una de las columnas de un DataFrame utilizando `ColumnTransformer()`. Recuerde  usar `BaseEstimator` y `TransformerMixin`.


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$


Consulte el siguiente [link](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer) si tiene dudas sobre la creación de custom transformers.

**Respuesta:**

In [17]:
class MinMax(BaseEstimator, TransformerMixin):

    def fit(self, X, y = None):
        # Búsqueda de los valores mínimos y máximos dentro de la columna
        self.min_x = X.min()
        self.max_x = X.max()
        return self

    def transform(self, X):
        X_scaled = X.copy()
        # Aplico la función minmax
        X_scaled = (X_scaled - self.min_x) / (self.max_x - self.min_x)
        return X_scaled

    def set_output(self, transform = 'default'):
        return self

#### 5.2 Incorporando MinMax al pipeline [5 puntos]

Ahora, usted decide agregar el escalamiento al pipeline, para lo que decide seguir los siguientes pasos:

- Agregar el paso `minmax` al pipeline `numeric_transformations`, haciendo uso de la clase creada. [1 punto]
- Defina el dataframe `df_minmax` aplicando el ColumnTransformer actualizado a los datos proporcionados por Mr. Cheems. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_minmax`. [1 punto]
- Reportar los cambios observados en la distribución de las variables.  [2 puntos]

>Nota: Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta:**

In [18]:
# Definicion las variables que pasarán por cada pipeline
numerical_columns = ('Price', 'Quantity')
categorical_columns = ('Invoice', 'StockCode', 'Description', 'InvoiceDate', 'Customer ID', 'Country')

# Pipeline para variables categóricas (standard)
categoric_transformation = Pipeline([('mode_imputer', SimpleImputer(strategy = 'most_frequent'))])

# Pipeline para variables numéricas (uso de KNN)
numeric_transformations = Pipeline([('iqr', IQR(lambda_param = 1.5)),
                                         ('knn_imputer', KNNImputer(n_neighbors = 3)),
                                         ('min_max', MinMax())])

# ColumnTransformer
column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', categoric_transformation, categorical_columns)
                                        ],
                                        verbose_feature_names_out = False)

# Mantener el formato de pandas
column_transformer.set_output(transform = 'pandas')

# Aplicamos ColumnTransformer a los datos
init = time.time()
df_minmax = column_transformer.fit_transform(df_retail)
end = time.time()
print('Tiempo de ejecución (en segundos):', end - init)

Tiempo de ejecución (en segundos): 1951.525897026062


In [ ]:
# Gráficos
print('Datos originales:')
explore_data(df_retail)

print('\nDatos después de aplicar MinMax:')
explore_data(df_minmax)

**Reportar los cambios observados en la distribución de las variables**

Los valores cambiaron el rango entre el que oscilan. Después de aplicar la función, ambas variables oscilan entre 0 a 1, teniendo una clara normalización de los valores. En esta parte ya se ve el efecto acumulativo de todos los pasos previos, por lo que ya no hay *outliers* y está la imputación elegida antes. La forma de la distribución se parece a las obtenidas antes, pero el rango es el que cambia, ya que se aplicó la clase `MinMax`. Es importante que para aplicar esta clase hay que sacar los *outliers* antes, ya que `MinMax` se vería afectada por valores atípicos, ensuciando así las distribuciones. 

También se ve la diferencia en los datos nulos: el dataframe que tiene aplicada la función MinMax ya no tiene valores nulos gracias al *pipeline* creado para eliminar valores e imputar nuevos en su primera etapa antes de la normalización.

### 6. Pregunta teórica [5 puntos]

<p align="center">
  <img width=300 src="https://file.coinexstatic.com/2023-09-19/166BAC031F222E5910954E7D7D0BC844.png">
</p>

Finalmente, explíquele a Mr. Cheems porqué es útil la creación de pipelines al momento de hacer Feature Engineering en Machine Learning.

**Respuesta:**

Crear pipelines en Python para el Feature Engineering en Machine Learning es muy útil porque organiza y simplifica el proceso de transformación de datos. En los ejercicios, por ejemplo, usamos pipelines para imputar datos faltantes y aplicar transformaciones como MinMax o IQR de forma ordenada. Esto no solo hace que el código sea más fácil de entender y mantener, sino que también permite aplicar las mismas transformaciones de manera consistente a diferentes conjuntos de datos. Además, los pipelines ayudan a evitar errores al automatizar pasos, asegurando que todas las transformaciones se apliquen correctamente antes de entrenar un modelo.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por el foro de U-cursos o por correo.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>